In [51]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
# from langchain_openai import OpenAI
# from langchain.chat_models import ChatOpenAI
# api_key = open("C://Users//Marcial//Desktop//desktop_openai.txt").read()
# model = ChatOpenAI(openai_api_key=api_key)

In [52]:
from langchain_google_genai import ChatGoogleGenerativeAI
# chat = ChatGoogleGenerativeAI(model="gemini-1.5-flash",convert_system_message_to_human=True, temperature=0)
chat = ChatGoogleGenerativeAI(model="gemini-1.5-flash",convert_system_message_to_human=True)

In [53]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [54]:
output_parser = CommaSeparatedListOutputParser()

In [55]:
format_instructions = output_parser.get_format_instructions();


In [56]:
print(format_instructions);


Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [57]:
reply = "one, two, three"
output_parser.parse("one, two, three")

['one', 'two', 'three']

In [58]:
human_template = '{request} {format_instructions}'
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [9]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

chat_prompt.format_prompt(request="give me 5 characteristics of dogs",
                   format_instructions = output_parser.get_format_instructions())

ChatPromptValue(messages=[HumanMessage(content='give me 5 characteristics of dogs Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`')])

In [10]:
request = chat_prompt.format_prompt(request="g, temperature=0ive me 5 characteristics of dogs",
                   format_instructions = output_parser.get_format_instructions()).to_messages()

In [ ]:
result = chat.invoke(request)

In [12]:
,temperature=0result.content

'loyalty, intelligence, playfulness, adaptability, affection \n'

In [13]:
output_parser.parse(result.content)

['loyalty', 'intelligence', 'playfulness', 'adaptability', 'affection']

In [14]:
from langchain.output_parsers import DatetimeOutputParser

In [15]:
output_parser = DatetimeOutputParser()

In [16]:
print(output_parser.get_format_instructions())

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 110-11-03T13:01:28.116833Z, 1581-04-26T14:10:39.333641Z, 1582-02-06T16:05:20.244492Z

Return ONLY this string, no other words!


In [17]:
template_text = "{request}\n{format_instructions}"
human_prompt=HumanMessagePromptTemplate.from_template(template_text)

In [18]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [19]:
print(chat_prompt.format(request="When was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ))

Human: When was the 13th Amendment ratified in the US?
Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 676-08-13T07:42:08.382294Z, 866-04-08T06:10:26.551731Z, 1249-01-05T10:03:51.375320Z

Return ONLY this string, no other words!


In [20]:
request = chat_prompt.format_prompt(request="What date was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ).to_messages();
                   

In [ ]:
result = chat.invoke(request);


In [22]:
result.content

'1865-12-18T00:00:00.000000Z\n'

In [23]:
output_parser.parse(str(result.content))

datetime.datetime(1865, 12, 18, 0, 0)

In [24]:
from langchain.output_parsers import OutputFixingParser

output_parser = DatetimeOutputParser()

misformatted = result.content

In [25]:
misformatted

'1865-12-18T00:00:00.000000Z\n'

In [27]:
new_parser = OutputFixingParser.from_llm(parser=output_parser, llm=chat)

In [28]:
new_parser.parse(misformatted)

datetime.datetime(1865, 12, 18, 0, 0)

In [29]:
system_prompt = SystemMessagePromptTemplate.from_template("You always reply to questions only in datetime patterns.")
template_text = "{request}\n{format_instructions}"
human_prompt=HumanMessagePromptTemplate.from_template(template_text)

In [30]:
chat_prompt = ChatPromptTemplate.from_messages([system_prompt,human_prompt]);


In [31]:
print(chat_prompt.format(request="When was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ))

System: You always reply to questions only in datetime patterns.
Human: When was the 13th Amendment ratified in the US?
Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1699-11-24T16:48:22.471214Z, 954-06-09T12:05:58.727333Z, 1772-06-29T22:19:08.943031Z

Return ONLY this string, no other words!


In [32]:
request = chat_prompt.format_prompt(request="What date was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ).to_messages()

In [ ]:
result = chat.invoke(request);


In [34]:
result.content


'1865-12-18T00:00:00.000000Z\n'

In [35]:
output_parser.parse(result.content)

datetime.datetime(1865, 12, 18, 0, 0)

In [36]:
from langchain.output_parsers import PydanticOutputParser

In [37]:
from pydantic import BaseModel, Field

In [38]:
class Scientist(BaseModel):
    
    name: str = Field(description="Name of a Scientist")
    discoveries: list = Field(description="Python list of discoveries")

In [39]:
query = 'Name a famous scientist and a list of their discoveries' 

In [40]:
parser = PydanticOutputParser(pydantic_object=Scientist)

In [41]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "Name of a Scientist", "title": "Name", "type": "string"}, "discoveries": {"description": "Python list of discoveries", "items": {}, "title": "Discoveries", "type": "array"}}, "required": ["name", "discoveries"]}
```


In [63]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query="Tell me about a famous scientist")

output = chat.invoke(_input.to_string());


print(parser.parse(output.content))
print()
print(_input.to_string())

/home/hiroshisakuma/projects/.py310env/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


name='Marie Curie' discoveries=['Radioactivity', 'Polonium', 'Radium']

Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "Name of a Scientist", "title": "Name", "type": "string"}, "discoveries": {"description": "Python list of discoveries", "items": {}, "title": "Discoveries", "type": "array"}}, "required": ["name", "discoveries"]}
```
Tell me about a famous scientist



In [64]:
type(parser.parse(output.content))

__main__.Scientist